In [14]:
import pickle
from itertools import product
from pathlib import Path

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.io import savemat

from hyppo.time_series import CCAX, MGCX, DcorrX, HsicX

In [47]:
def generate_phi(phi=0.5, d=1):
    out = np.eye(d)
    denom = 1 / np.arange(1, d + 1)
    out = out * denom * phi

    return out


def generate_var(n, d, sigma=1, lag=1, phi=0.5):
    epsilons = np.random.normal(0, sigma, (n, d))
    etas = np.random.normal(0, sigma, (n, d))

    x = epsilons
    y = etas

    phis = generate_phi(phi, d)

    for t in range(lag, n):
        x[t] = phis @ y[t - lag] + epsilons[t]
        y[t] = phis @ x[t - lag] + etas[t]

    return x, y


def generate_indep_var(n, d, sigma=1, lag=1, phi=0.5):
    epsilons = np.random.normal(0, sigma, (n, d))
    etas = np.random.normal(0, sigma, (n, d))

    x = epsilons
    y = etas

    phis = generate_phi(phi, d)

    for t in range(lag, n):
        x[t] = phis @ x[t - lag] + epsilons[t]
        y[t] = phis @ y[t - lag] + etas[t]

    return x, y


def generate_data(
    n, d, lag, phi, fname, output_dir="./data/", reps=300, generate_func=generate_var
):
    X_full = np.zeros((reps, n, d))
    Y_full = np.zeros((reps, n, d))
    for s in range(reps):
        np.random.seed(s)
        X_full[s], Y_full[s] = generate_func(n, d, lag=lag, phi=phi)

    # Save simulated output.
    output = {"X": X_full, "Y": Y_full}
    p = Path(output_dir)
    if not p.is_dir():
        p.mkdir(parents=True)

    filename = p / f"{fname}_data.pkl"
    file = open(filename, "wb")
    pickle.dump(output, file)
    file.close()

    # Save to MATLAB format as well.
    savemat(p / f"{fname}_data.mat", {"X_full": X_full, "Y_full": Y_full})


def run_experiment(test, n, d, reps=300, file=""):
    with open(file, "rb") as f:
        dat = pickle.load(f)
    # X = dat["X"]
    # Y = dat["Y"]

    pvals = []

    for seed in range(reps):
        np.random.seed(seed)

        x = dat["X"][seed, :n, :d]
        y = dat["Y"][seed, :n, :d]

        print(x.shape, y.shape)
        res = test.test(x, y, reps=1000, workers=1)
        pvals.append(res[1])

    rejects = np.array(pvals) <= 0.05

    return np.mean(rejects), np.std(rejects)

In [52]:
generate_data(100, 100, 1, 0.5, "indep_var1", generate_func=generate_indep_var)
generate_data(100, 100, 1, 0.5, "linear_var1", generate_func=generate_var)

In [57]:
tests = [
    ["dcorr", DcorrX(max_lag=1)],
    ["cca", CCAX(max_lag=1)],
    ["hsic", HsicX(max_lag=1)],
    ["mgc", MGCX(max_lag=1)],
]

sigma = 1
n = 100
ds = np.arange(0, 101, 10)
ds[0] = 1
lag = 1
reps = 300

args = list(product(tests, ds))

In [ ]:
res = Parallel(-1, verbose=10)(
    delayed(run_experiment)(test, n, d, reps=reps, file="./data/linear_var1_data.pkl")
    for (_, test), d in args
)

df = [[a[0][0]] + [a[1]] + [*b] for a, b in zip(args, res)]
df = pd.DataFrame(df, columns=["test", "d", "power", "std"])
df.to_csv("./linear_var.csv", index=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


In [ ]:
res = Parallel(-1, verbose=10)(
    delayed(run_experiment)(test, n, d, reps=reps, file="./data/indep_var1_data.pkl")
    for (_, test), d in args
)

df = [[a[0][0]] + [a[1]] + [*b] for a, b in zip(args, res)]
df = pd.DataFrame(df, columns=["test", "d", "power", "std"])
df.to_csv("./indep_var.csv", index=False)